In [77]:
'''
Author: Suryakant Upadhayay
Purpose: This script is a sample demonstration to create and validate cloud optimized GeoTiff (COG)  file using:
    1. From RGB GeoTiff (created here using any 3 bands of the provided remote sensing data)
    2. From multi/hyper spectral bands remote sensing data
    
'''

from osgeo import gdal, osr
import os
import io
import numpy as np
import glob

def createRgbTiff(newRasterFilePath,rasterOrigin,pixelWidth, pixelHeight, array, proj,format='GTiff'):

    bands = array.shape[0]
    rows = array.shape[1]
    cols = array.shape[2]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]
    driver = gdal.GetDriverByName(format)
    #Here is where I assign three bands to the raster with int 3
    options = ['PHOTOMETRIC=RGB', 'PROFILE=GeoTIFF']
    outRaster = driver.Create(newRasterFilePath, cols, rows, bands, gdal.GDT_UInt16, options=options)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    #outband = outRaster.GetRasterBand(1)
    #outband.WriteArray(array)
    for band in range(bands):
        outRaster.GetRasterBand(band+1).WriteArray( array[band, :, :] )

    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(proj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())

    outRaster.GetRasterBand(1).SetColorInterpretation(gdal.GCI_RedBand)
    outRaster.GetRasterBand(2).SetColorInterpretation(gdal.GCI_GreenBand)
    outRaster.GetRasterBand(3).SetColorInterpretation(gdal.GCI_BlueBand)
    outRaster.FlushCache()
    print('RGB COG creation successful')

def createCOG(newRasterfn,array, rasterOrigin, pixelWidth, pixelHeight, proj):
    cols = None
    rows = None
    nbands = None
    if(len(array.shape)==3):
        cols = array.shape[2]
        rows = array.shape[1]
        nbands = array.shape[0]
    else:
        cols = array.shape[1]
        rows = array.shape[0]
        nbands = 1
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]
    driver = gdal.GetDriverByName('MEM')
    data_set = driver.Create('', cols, rows, nbands,
                             gdal.GDT_Float32)
    data_set.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    if(nbands==1):
        data_set.GetRasterBand(1).WriteArray(array)
    else:
        for i in range(nbands):
            data_set.GetRasterBand(i + 1).WriteArray(array[i])

    data_set_CRS = osr.SpatialReference()
    data_set_CRS.ImportFromEPSG(proj)
    data_set.SetProjection(data_set_CRS.ExportToWkt())
    data = None
    data_set.BuildOverviews("NEAREST", [2, 4, 8, 16, 32, 64])

    driver = gdal.GetDriverByName('GTiff')
    data_set2 = driver.CreateCopy(newRasterfn, data_set,
                                  options=["COPY_SRC_OVERVIEWS=YES",
                                           "TILED=YES",
                                           "COMPRESS=LZW"])
    data_set = None
    data_set2 = None


def readSampleRgbBands():
    print("Reading raster bands for RGB")
    #reading and combining any 3 tiffs to create a raster RGB band array, in this example reading band index 7, 8, 9
    rgb_array = np.stack([gdal.Open(tiffs[7]).ReadAsArray(),
                          gdal.Open(tiffs[8]).ReadAsArray(),
                          gdal.Open(tiffs[9]).ReadAsArray()])
    print("Reading raster bands successful")
    return rgb_array

if __name__ == "__main__":
     
    #path to the folder which contains different bands data i.e tif files in this case    
    path = r'C:\Users\Suryakant Upadhayay\Downloads\EO1H1430452010208110Kt_1GST\EO1H1430452010208110Kt'
    tiffs = glob.glob(os.path.join(path,"*.tif*"))  #getting list of all the available tiffs in the folder

    #getting geotransforms and crs
    transforms = gdal.Open(tiffs[0]).GetGeoTransform()
    rasterOrigin = [transforms[0],transforms[3]]
    pixelWidth = transforms[1]
    pixelHeight = transforms[5]
    proj = int(osr.SpatialReference(wkt=gdal.Open(tiffs[0]).GetProjection()).GetAttrValue('AUTHORITY',1))
    print("CRS EPSG Code: ", proj)
    
    ##creating and validating COG using RGB first    
    sampleRGBArray = readSampleRgbBands()
    #output RGB COG file path
    sampleOutputPath = r'C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB_2.tif'
    createRgbTiff(sampleOutputPath,rasterOrigin,pixelWidth, pixelHeight, sampleRGBArray, proj)
    print("Creating COG from sample RGB")
    newRasterfn = r'C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB_COG_2.tif'
    array = gdal.Open(r'C:\Users\Suryakant Upadhayay\Downloads\NDVI.tiff').ReadAsArray()#gdal.Open(sampleOutputPath).ReadAsArray()
    print(array.shape)
    createCOG(newRasterfn,array, rasterOrigin, pixelWidth, pixelHeight, proj)
    print("Validating created Sample RGB COG")
    #there is script Test_COG.py to validate COG, calling the script with script and COG paths as arguments
    %run -i "C:\Users\Suryakant Upadhayay\Downloads\Test_COG.py" "C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB_COG_2.tif"
    
    ##creating and validating COG using multi/hyper spectral bands remote sensing data
    print("Reading multi/hyper spectral bands")
    arrs = []
    for tiff in tiffs:
        B = gdal.Open(tiff)
        b = B.ReadAsArray()
        arrs.append(b)
    
    array = np.stack(arrs)
    print(array.shape, array.ndim)
    print("Creating COG from multi/hyper spectral bands")
    newRasterfn = r'C:\Users\Suryakant Upadhayay\Downloads\Sample_COG_2.tif'
    createCOG(newRasterfn,array, rasterOrigin, pixelWidth, pixelHeight, proj)
    print("Validating created Sample COG")
    #there is script Test_COG.py to validate COG, calling the script with script and COG paths as arguments
    %run -i "C:\Users\Suryakant Upadhayay\Downloads\Test_COG.py" "C:\Users\Suryakant Upadhayay\Downloads\Sample_COG_2.tif"
    
    
    print("Finished")



CRS EPSG Code:  32644
Reading raster bands for RGB
Reading raster bands successful
RGB COG creation successful
Creating COG from sample RGB
(3481, 1021)
Validating created Sample RGB COG
C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB_COG_2.tif is a valid cloud optimized GeoTIFF

The size of all IFD headers is 2366 bytes
Reading multi/hyper spectral bands
(242, 3521, 1061) 3
Creating COG from multi/hyper spectral bands
Validating created Sample COG
C:\Users\Suryakant Upadhayay\Downloads\Sample_COG_2.tif is a valid cloud optimized GeoTIFF

The size of all IFD headers is 12912 bytes
Finished


In [83]:
'''
Author: Suryakant Upadhayay
Purpose: This script reads data at a point from COG. This can be utilized over http to read data from COG
         This concept can be further extended and utilized to read and create a image, in a region/bounding box from COG, of a particular band,
         using gdal and numpy arrays over http
'''
import rasterio
from osgeo import gdal
def readDataValueFromCOG(geom, raster_path, band):   
        try:
            with rasterio.open(raster_path, mode='r') as src:
                x = geom['lng'] 
                y = geom['lat']          
                # Sample the raster at the given coordinates
                value_gen = src.sample([(x, y)], indexes=[band])
                value = value_gen.__next__().item(0)
                print("Band:",band,"Value:",value,"Location:",geom)
                return ({"data": value,"error": None})
        except rasterio.RasterioIOError as e:
            print('error', str(e))
            return ({"data": None,"error": str(e)})

cogPath  = r'C:\Users\Suryakant Upadhayay\Downloads\Sample_COG_2.tif'
#getting geotransforms and crs
transforms = gdal.Open(cogPath).GetGeoTransform()
print("Transform : ",transforms)
proj = int(osr.SpatialReference(wkt=gdal.Open(tiffs[0]).GetProjection()).GetAttrValue('AUTHORITY',1))
print("CRS EPSG Code: ", proj)
samplePoint = {'lng':453900,'lat':2490000}
band = 140
readDataValueFromCOG(samplePoint, cogPath, band)
band = 1
readDataValueFromCOG(samplePoint, cogPath, band)

Transform :  (453900.0, 29.97172478793591, 0.0, 2490000.0, 0.0, -29.991479693268957)
CRS EPSG Code:  32644
Band: 140 Value: 0.0 Location: {'lng': 453900, 'lat': 2490000}
Band: 1 Value: 0.0 Location: {'lng': 453900, 'lat': 2490000}


{'data': 0.0, 'error': None}